In [406]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.optimize import curve_fit


from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [407]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=3)
import qutip as qt
import matplotlib.style as style
style.use('S:\Connie\prx.mplstyle')

# Parameters

In [427]:
use_init_2q_state = True
plot = True
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412091502_init_rhoMLE_ZZ_2Q_01.npz'
n_tomo_corrected_filepath = "S:\QRAM\qram_4QR2\data\data_241025\\202412081553_ntomocorrected_3Q_023.npz"
save_plot_path = 'S:\\QRAM\\qram_4QR2\\plots\\tomo'



evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
tomo_qubits = [0, 2, 3]
phis = [None, None, None] 

phis[0] = np.linspace(0, 360, 20)
phis[1] = np.linspace(0, 360, 20)
phis[2] = np.linspace(0, 360, 20)

color_routing = '#ff7f50'
color_signal = '#468499'
color_q2 = '#E16F8F'
color_basis = '#66CDAA'
color_vec = []

rho_qram = {'rho_base': {},
            'rho_init': {},
            'rho_ZZ': {},
            'fidelity_base': {},
            'fidelity_ZZ': {},
            'purity_base': {},
            'purity_ZZ': {},
            'rho_base_rotated': {},
            'rho_ZZ_rotated': {},
            'fidelity_base_rotated': {},
            'fidelity_ZZ_rotated': {},
            'fidelity_base_avg': {},
            'fidelity_ZZ_avg': {},
            }



In [428]:
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

In [429]:
def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    # SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]
    SI_to_SLR = [psis['000'], psis['010'], psis['100'], psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)


### Import 2q tomo

In [430]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
# rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202411051014_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_241025\202412091502_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


In [426]:
_tomo_analysis = TomoAnalysis(nb_qubits=2)
for init_state in rho_MLE_ZZ_dict.keys():
    _tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ_dict[init_state], title=f'Init {init_state}', cmax=0.5, size=(6,6), no_show=True)

### Load evolv mats

In [431]:
print('Will save evol mats to path', evol_mats_path)
qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'

qA, qB, qC = tomo_qubits
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})

Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats
Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


In [432]:
print(f"Using n_tomo_corrected_dict from file {n_tomo_corrected_filepath}")
n_tomo_corrected_dict = dict()

# n_tomo_corrected_dict.update({"saved_files":saved_files})
with np.load(n_tomo_corrected_filepath, allow_pickle=True) as npzfile:
    for key in npzfile.keys():
        n_tomo_corrected_dict.update({key:npzfile[key]})

init_states = n_tomo_corrected_dict["init_states"]

if 'saved_files' in init_states: init_states = init_states[:-1]

print(f'n_tomo_corrected_dict retrieved with init_states\n{init_states}')
all_configs = n_tomo_corrected_dict["all_configs"].item() #[0]
# print(all_configs.keys())


cfg = AttrDict(all_configs[init_states[0]])
play_pulses = cfg.expt.play_pulses
tomo_qubits = cfg.expt.tomo_qubits

print("play_pulses", play_pulses)

Using n_tomo_corrected_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412081553_ntomocorrected_3Q_023.npz
n_tomo_corrected_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 4, 3]


In [433]:
for init_state in init_states:
    print('init state (Q0 (switch) Q1 (input)):', init_state)
    n_tomo_corrected = n_tomo_corrected_dict[init_state]
    
    if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
    else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()
    rho_id = get_qram_qSLR_state_from_rho(init_rho)

    # ------------------------------ #
    # Base MLE
    # ------------------------------ #
    
    rho_MLE_base = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=False,
    )
 
    fid_base = qt.fidelity(qt.Qobj(rho_MLE_base, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_base = np.real(np.trace(rho_MLE_base @ rho_MLE_base))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    # print(f'Fidelity (base): {fid_base}')
    # print(f'Purity (base): {purity_base}')
    # print(f'Target purity: {purity_id}')
    
    # if saveplots: savetitle = saved_file[:-3]+f'_base{save_append}.svg'
    # else: savetitle = None
    # show_plot_rho_2d(rho_MLE_base, rho_id, title=f'Base MLE (Q{tomo_qubits[0]}, Q{tomo_qubits[1]}, Q{tomo_qubits[2]}), Init {init_state}, Play {play_pulses}', savetitle=savetitle, cmax=0.5)
    
    # rhos_base.append(rho_MLE_base)
    # fidelities_base.append(fid_base)
    # purities_base.append(purity_base)
    
    # tomo_analysis.show_plot_rho_2d(rho_MLE_base, rho_id.full(), title=f'Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6), savetitle=save_plot_path+'\\' +_init_state+'_base.pdf', no_show=True)
    
    
    rho_qram['rho_base'][init_state] = rho_MLE_base
    rho_qram['fidelity_base'][init_state] = fid_base
    rho_qram['purity_base'][init_state] = purity_base
    rho_qram['rho_init'][init_state] = rho_id
        
    rho_MLE = rho_MLE_base

    # ------------------------------ #
    # MLE with ZZ
    # ------------------------------ #

    rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_corrected,
        n_calib=None,
        correct_readout=False,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats
    )

    fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
    purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
    # print(f'Fidelity (with ZZ correction): {fid_ZZ}')
    # print(f'Purity (with ZZ correction): {purity_ZZ}')
    # print(f'Target purity: {purity_id}')
    
    
    _init_state = init_state.replace('|', '')
    _init_state = _init_state.replace('>', '')



    # tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ, rho_id.full(), title=f'Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6), savetitle=save_plot_path+'\\' +_init_state+'_ZZ.pdf', no_show=True)
    
    rho_qram['rho_ZZ'][init_state] = rho_MLE_ZZ
    rho_qram['fidelity_ZZ'][init_state] = fid_ZZ
    rho_qram['purity_ZZ'][init_state] = purity_ZZ

init state (Q0 (switch) Q1 (input)): |0>|0>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0>|1>
constructing final state on |switch, out1, out2>


 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1234ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1239

init state (Q0 (switch) Q1 (input)): |0>|0+1>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0>|0+i>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |1>|0>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |1>|1>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |1>|0+1>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |1>|0+i>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0+1>|0>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0+1>|1>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0+1>|0+1>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0+1>|0+i>
constructing final state on |switch, out1, out2>
init state (Q0 (switch) Q1 (input)): |0+i>|0>
cons

## Rotations

### From base

In [434]:
fid_grid_base = []

In [435]:
for init_state in init_states:
    _rho_base = rho_qram['rho_base'][init_state]
    _rho_id = rho_qram['rho_init'][init_state]
    rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(qt.Qobj(_rho_base, dims=rho_id.dims), qt.Qobj(_rho_id, dims=id3q.dims), phis=phis, progress=True)
    rho_qram['rho_base_rotated'][init_state] = rho_MLE_rot
    rho_qram['fidelity_base_rotated'][init_state] = best_fid
    print('State:', init_state)
    print(f'Best fidelity for base rotated: {best_fid}')
    print(f'compared to fidelity without rotation: {rho_qram["fidelity_base"][init_state]}')
    
    fid_grid_base.append(fids_grid)
    
    _init_state = init_state.replace('|', '')
    _init_state = _init_state.replace('>', '')
    

    # tomo_analysis.show_plot_rho_2d(rho_MLE_rot.full(), _rho_id.full(), title=f'Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6), savetitle=save_plot_path+'\\' +_init_state+'_opt_angle_base.pdf', no_show=True)
    


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 0.03392528653565208
State: |0>|0>
Best fidelity for base rotated: 0.9873463243349291
compared to fidelity without rotation: 0.9870070714695726


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 0.1420038016305747
State: |0>|1>
Best fidelity for base rotated: 0.9558398200243213
compared to fidelity without rotation: 0.9544197820080156


  0%|          | 0/8000 [00:00<?, ?it/s]

...

Improved fidelity by (%) 4.899849163847502
State: |0+i>|1>
Best fidelity for base rotated: 0.9678841571977551
compared to fidelity without rotation: 0.9188856655592801


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 54.218997870617855
State: |0+i>|0+1>
Best fidelity for base rotated: 0.9644675236059627
compared to fidelity without rotation: 0.42227754489978414


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 48.974087922021056
State: |0+i>|0+i>
Best fidelity for base rotated: 0.96299024305662
compared to fidelity without rotation: 0.47324936383640936


In [436]:
# remove 11 from the init states
check_init_states = init_states
# check_init_states = init_states[init_states != '|1>|1>']


In [437]:
fids_grids_reshaped = []
for i_state in range(len(init_states)):
    grid_shape = fid_grid_base[i_state].shape
    # print(grid_shape)
    fids_grid = np.copy(fid_grid_base[i_state]).tolist()
    # print(init_states[i_state], grid_shape)
    if grid_shape[0] == 1:
        for i in range(grid_shape[1]-1):
            fids_grid.append(fids_grid[0])
    # print(np.array(fids_grid).shape)
    fids_grids_reshaped.append(fids_grid)
    
fids_grids_reshaped = np.array(fids_grids_reshaped)
print(fids_grids_reshaped.shape)


best_avg_fid = 0
best_fids = []
best_phis = [0, 0, 0]
best_phi_indices = [0, 0, 0]
for iphi0, phi0 in enumerate(phis[0]):
    for iphi1, phi1 in enumerate(phis[1]):
        for iphi2, phi2 in enumerate(phis[2]):
            avg_fid = 0
            for i_state, init_state in enumerate(init_states):
                if init_state not in check_init_states: continue
                avg_fid += fids_grids_reshaped[i_state][iphi0, iphi1, iphi2]
            avg_fid /= len(check_init_states)
            if avg_fid > best_avg_fid:
                best_phis = [phi0, phi1, phi2]
                best_phi_indices = [iphi0, iphi1, iphi2]
                best_avg_fid = avg_fid
                
                
                
                
for id, init_state in enumerate(init_states):
    _fid_avg = fids_grids_reshaped[id][best_phi_indices[0], best_phi_indices[1], best_phi_indices[2]]
    rho_qram['fidelity_base_avg'][init_state] = _fid_avg
                
                


(16, 20, 20, 20)


In [438]:
print('Average fidelity:', best_avg_fid)
print('Best fidelities:', np.mean(list(rho_qram['fidelity_base_rotated'].values())))

Average fidelity: 0.9381422226418243
Best fidelities: 0.9604561066102315


### Plotting

In [439]:
color_vec = []

for i_state, init_state in enumerate(init_states):
    
    _init_state = init_state.replace('|', '')
    q0, q1 = _init_state[:-1].split('>')
    if q0 == '0+1' or q0=='0+i':
        if q1 != '0+1' and q1 !='0+i':
            color_vec.append(color_routing)
        else:
            color_vec.append(color_q2)

    else: 
        if q1 == '0+1' or q1=='0+i':
            color_vec.append(color_signal)
        else:
            color_vec.append(color_basis)


In [440]:
nb_color = []

# sort the data so that the color are together and in the color_basis/color_signal/color_routing/color_q2 order

idx_basis = [i for i, color in enumerate(color_vec) if color == color_basis]
idx_signal = [i for i, color in enumerate(color_vec) if color == color_signal]
idx_routing = [i for i, color in enumerate(color_vec) if color == color_routing]
idx_q2 = [i for i, color in enumerate(color_vec) if color == color_q2]

idx_order = idx_basis + idx_signal + idx_routing + idx_q2

print(idx_order)


[0, 1, 4, 5, 2, 3, 6, 7, 8, 9, 12, 13, 10, 11, 14, 15]


In [441]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))

x_labels = []
y_best = []
y_avg = []

color_plot = []
for i in idx_order:
    init_state = init_states[i]
    _init_state = init_state.replace('>', '\\rangle')
    x_labels.append(f'${_init_state}$')
    y_best.append(rho_qram['fidelity_base_rotated'][init_state])
    y_avg.append(rho_qram['fidelity_base_avg'][init_state])
    color_plot.append(color_vec[i])
    
ax.bar(x_labels, y_best, color=color_plot, alpha=0.8, edgecolor=color_plot, linewidth=1, fill=True)
ax.bar(x_labels, y_avg, color='white', edgecolor=color_plot, linewidth=1, fill=False)

# tilt the x labels by 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

ax.set_ylabel('Fidelity')


ax.set_ylim(0.75, 1.01)

fig.tight_layout()
fig.show()


### From ZZ

In [442]:
fid_grid_ZZ = []

for init_state in init_states:
    _rho_base = rho_qram['rho_ZZ'][init_state]
    _rho_id = rho_qram['rho_init'][init_state]
    rho_MLE_rot, best_phis, best_fid, fids_grid = tomo_analysis.opt_virtualZ_MLE(qt.Qobj(_rho_base, dims=rho_id.dims), qt.Qobj(_rho_id, dims=id3q.dims), phis=phis, progress=True)
    rho_qram['rho_ZZ_rotated'][init_state] = rho_MLE_rot
    rho_qram['fidelity_ZZ_rotated'][init_state] = best_fid
    print('State:', init_state)
    print(f'Best fidelity for base rotated: {best_fid}')
    print(f'compared to fidelity without rotation: {rho_qram["fidelity_base"][init_state]}')
    
    _init_state = init_state.replace('|', '')
    _init_state = _init_state.replace('>', '')
    print('fid_grid_ZZ', fids_grid.shape)
    fid_grid_ZZ.append(fids_grid)


    # tomo_analysis.show_plot_rho_2d(rho_MLE_rot.full(), _rho_id.full(), title=f'Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6), savetitle=save_plot_path+'\\' +_init_state+'_opt_angle_ZZ.pdf', no_show=True)

  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 0.020341953724456463
State: |0>|0>
Best fidelity for base rotated: 0.9874624853880273
compared to fidelity without rotation: 0.9870070714695726
fid_grid_ZZ (20, 20, 20)


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 0.06255913659150014
State: |0>|1>
Best fidelity for base rotated: 0.9557620736018843
compared to fidelity without rotation: 0.9544197820080156
fid_grid_ZZ (20, 20, 20)


  0%|          | 0/8000 [00:00<?, ?it/s]

...

Improved fidelity by (%) 1.4882605307742103
State: |0+i>|1>
Best fidelity for base rotated: 0.9682061401810317
compared to fidelity without rotation: 0.9188856655592801
fid_grid_ZZ (20, 20, 20)


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 47.86560060455016
State: |0+i>|0+1>
Best fidelity for base rotated: 0.943143975514787
compared to fidelity without rotation: 0.42227754489978414
fid_grid_ZZ (20, 20, 20)


  0%|          | 0/8000 [00:00<?, ?it/s]

Improved fidelity by (%) 42.887767182864366
State: |0+i>|0+i>
Best fidelity for base rotated: 0.9451464537728597
compared to fidelity without rotation: 0.47324936383640936
fid_grid_ZZ (20, 20, 20)


In [443]:
check_init_states = init_states

In [444]:
fids_grids_reshaped = []
for i_state in range(len(init_states)):
    grid_shape = fid_grid_ZZ[i_state].shape
    # print(grid_shape)
    fid_grid = np.copy(fid_grid_ZZ[i_state]).tolist()
    # print(init_states[i_state], grid_shape)
    if grid_shape[0] == 1:
        for i in range(grid_shape[1]-1):
            fid_grid.append(fid_grid[0])
    # print(np.array(fids_grid).shape)
    fids_grids_reshaped.append(fid_grid)
    
fids_grids_reshaped = np.array(fids_grids_reshaped)
print(fids_grids_reshaped.shape)


best_avg_fid_zz = 0
best_fids_zz = []
best_phis_zz = [0, 0, 0]
best_phi_indices_zz = [0, 0, 0]
for iphi0, phi0 in enumerate(phis[0]):
    for iphi1, phi1 in enumerate(phis[1]):
        for iphi2, phi2 in enumerate(phis[2]):
            avg_fid = 0
            for i_state, init_state in enumerate(init_states):
                if init_state not in check_init_states: continue
                avg_fid += fids_grids_reshaped[i_state][iphi0, iphi1, iphi2]
            avg_fid /= len(check_init_states)
            if avg_fid > best_avg_fid_zz:
                best_phis_zz = [phi0, phi1, phi2]
                best_phi_indices_zz = [iphi0, iphi1, iphi2]
                best_avg_fid_zz = avg_fid
    

for id, init_state in enumerate(init_states):
    
    _fid_avg = fids_grids_reshaped[id][best_phi_indices_zz[0], best_phi_indices_zz[1], best_phi_indices_zz[2]]
    rho_qram['fidelity_ZZ_avg'][init_state] = _fid_avg


(16, 20, 20, 20)


In [445]:
print('Average fidelity:', best_avg_fid_zz)
print('Best fidelities:', np.mean(list(rho_qram['fidelity_ZZ_rotated'].values())))

Average fidelity: 0.9354889418831361
Best fidelities: 0.9599780267203624


In [446]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3.5))

y_best_zz = []
y_avg_zz = []


for i in idx_order:
    init_state = init_states[i]
    y_best_zz.append(rho_qram['fidelity_ZZ_rotated'][init_state])
    y_avg_zz.append(rho_qram['fidelity_ZZ_avg'][init_state])
    color_plot.append(color_vec[i])
    
ax.bar(x_labels, y_best_zz, color=color_plot, alpha=0.5, edgecolor=color_plot, linewidth=1.5, fill=False, label = 'ZZ corrected best')
ax.bar(x_labels, y_best, color=color_plot, alpha=0.5, edgecolor='black', linewidth=1.5, fill=True, label = 'Base best')
ax.bar(x_labels, y_avg_zz, color=color_plot, alpha=0.5, edgecolor='black', linewidth=0, fill=True, label = 'ZZ corrected avg')
ax.bar(x_labels, y_avg, color=color_plot, alpha=.4, edgecolor='black', linewidth=1.5, linestyle='--', 
       fill=False, label = 'Base avg')

ax.legend()



text= r'$F_\mathrm{zz, best} = $' + '%.3f'%(np.mean(list(rho_qram['fidelity_ZZ_rotated'].values()))*100) + '%'
text+= '\n'
text+= r'$F_\mathrm{base, best} = $' + '%.3f'%(np.mean(list(rho_qram['fidelity_base_rotated'].values()))*100) + '%'
text+= '\n'
text+= r'$F_\mathrm{zz, avg} = $' + '%.3f'%(best_avg_fid_zz*100) + '%'
text+= '\n'
text+= r'$F_\mathrm{base, avg} = $' + '%.3f'%(best_avg_fid*100) + '%'



# f'{_F:.4f}\n'
# text += r'F_{base, best}:' + f'{np.mean(list(rho_qram["fidelity_base_rotated"].values())):.3f}\n'
# text += r'F_{zz, avg}:' + f'{best_avg_fid_zz:.4f}\n'
# text += r'F_{base, avg}:' + f'{best_avg_fid:.4f}\n'

# ax.text(0.5, 0.5, text)
ax.text(0.5, 0.5, text, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='white', alpha=0.5), fontsize=12)

# tilt the x labels by 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)

ax.set_ylabel('Fidelity')

ax.set_ylim(0.75, 1.01)


plot_title = f'Fidelity comparison, Play {play_pulses}, Measure init states: {use_init_2q_state}'

fig.suptitle(plot_title)
fig.tight_layout()
fig.show()

savename = save_plot_path + f'\\fidelity_comparison_{play_pulses}_init_{use_init_2q_state}.pdf'

fig.savefig(savename, bbox_inches='tight')



### Paper figures 

In [450]:
# for all y average the fidelities with the same color
y_qq_avg = {}
y_qq_avg['Cr-Cs'] = np.mean([y_avg[i] for i in idx_basis])
y_qq_avg['Cr-Qs'] = np.mean([y_avg[i] for i in idx_signal])
y_qq_avg['Qr-Cs'] = np.mean([y_avg[i] for i in idx_routing])
y_qq_avg['Qr-Qs'] = np.mean([y_avg[i] for i in idx_q2])


_color_vec = [color_basis, color_signal, color_routing, color_q2]


fig, ax = plt.subplots(1, 1, figsize=(3, 3))

x_labels = list(y_qq_avg.keys())


# break the line between routing and signal


# for i in len(x_labels):
    


ax.bar(x_labels, y, color=_color_vec, alpha=0.5, edgecolor=_color_vec, linewidth=1.5, fill=True)

ax.set_ylabel('Fidelity')

ax.set_ylim(0.75, 1.01)



fig.show()